# Model evaluation
In some cases, we may not know the number of segments or we may wish to compare a power law against other types of rating models.
For these cases, we can use information criteria to select the best model.

In [ ]:
# load tutorial data
from ratingcurve import data
%load_ext autoreload
%autoreload 2
%xmode minimal
#suppress warnings and errors


import pymc as pm
import arviz as az
from ratingcurve.ratingmodel import PowerLawRating

from ratingcurve import data

df = data.load('green channel')

Fit the data to ratings with 1 to 4 segments and determine which is best.

In [ ]:
%%capture
# OUtput supressed, this will print "Finished" after running each of the four models

segments = [1, 2, 3, 4]
traces = []
for segment in segments:
    powerrating = PowerLawRating(q=df['q'],
                             h=df['stage'], 
                             q_sigma=df['q_sigma'],
                             segments=segment,
                             prior={'distribution':'uniform'})
    
    trace = powerrating.fit(n=100_000)
    traces.append(pm.compute_log_likelihood(trace)) # Add arg to compute log likelihood

now use `arviz.compare` to format the output

In [ ]:
# this model will generate warnings about the LOO
import warnings; warnings.filterwarnings('ignore')

compare_dict = {f'{i} segment': traces[i-1] for i in segments}
az.compare(compare_dict, ic='LOO')

As we expected, the 2-segment model was ranked highest.

# Residuals
In practice, it can be helpful to plot to rating error
(the deviations between the rating fit and the discharge observations).
Here is a demonstration of how.

In [ ]:
import matplotlib.pyplot as plt

segments = 2
powerrating = PowerLawRating(q=df['q'],
                             h=df['stage'], 
                             q_sigma=df['q_sigma'],
                             segments=segments,
                             prior={'distribution':'uniform'})

In [ ]:
with powerrating:
    mean_field = pm.fit(method='advi', n=150_000)
    trace = mean_field.sample(5000)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,7), sharey=True)

powerrating.plot(trace, ax[0])
powerrating.plot_residuals(trace, ax[1])


plt.subplots_adjust(wspace=0.1)
ax[1].set_ylabel('')